In [ ]:
#!/usr/bin/env python
import ROOT, sys, os
from ROOT import std

from larlite import larlite
from larcv import larcv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.path as path
colormap=['blue','red','magenta','green','orange','yellow','pink']
nc=len(colormap)
from ROOT import geo2d
from ROOT import cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

In [ ]:
iom=larcv.IOManager(larcv.IOManager.kBOTH)        
iom.reset()
iom.set_verbosity(0)                                                                          
iom.add_in_file("mcc7_bnb_detsim_to_larcv_hires_v00_p00_out_0000.root")
#iom.add_in_file("out_0000_0099.root")
iom.set_out_file("/tmp/trash.root")
iom.initialize()
iom.read_entry(10)#8

larbysimg=larcv.LArbysImage()
cfg=larcv.CreatePSetFromFile("unit.fcl","LArbysImage")
larbysimg.configure(cfg)
larbysimg.initialize()

In [ ]:
larbysimg.process(iom)

In [ ]:
geo2d.PyDraw()

In [ ]:
pygeo   = geo2d.PyDraw()

mgr=larbysimg.Manager()
img_v = []

xranges=((300,900),(400,700),(450,700))
yranges=((700,1275),(750,900),(1350,1600))

plane=0
for mat in mgr.InputImages():
    #img_v.append(pygeo.image(mat).transpose())
    img_v.append(pygeo.image(mat))
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    fig,ax=plt.subplots(figsize=(12,12))
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    ax.set_ylim(yranges[plane][0],yranges[plane][1])
    ax.set_xlim(xranges[plane][0],xranges[plane][1])
    plt.show()
    plane+=1

dm=mgr.DataManager()

In [ ]:
for plane in xrange(3):
    print 'Plane',plane
    fig,ax=plt.subplots(figsize=(20,10))

    #Get the image
    shape_img = img_v[plane]
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(np.max(nz_pixels[0])-10,np.min(nz_pixels[0])+10)
    ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    #ax.set_ylim(900,1100)
    #ax.set_xlim(400,700)

    data=dm.Data(0)

    all_p0=mgr.Clusters(plane,0)

    for ctor in all_p0:
        pts=np.array([[pt.x,pt.y] for pt in ctor._contour])
        plt.plot(pts[:,0],pts[:,1],'-o')

    ax.set_aspect(1.0)
    plt.tight_layout()
    ax.set_title("Plane {}".format(plane),fontweight='bold',fontsize=40)
    plt.show()

In [ ]:
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(20,20))

    #Get the image
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    #ax.set_ylim(650,700)
    #ax.set_xlim(275,325)

    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    data=dm.Data(1)
    
    compound_v = data._raw_cluster_vv[plane]
    
    for compound in compound_v.get_cluster():
        for atom in compound.get_atoms():
            pts = np.array([[pt.x,pt.y] for pt in atom._ctor])
            plt.plot(pts[:,0],pts[:,1],'-o')
        for defect in compound.get_defects():
            ax.plot([0,900],[defect._split_line.y(0),defect._split_line.y(900)])

    ax.set_aspect(2.0)
    ax.set_title("ALL Contours")
    plt.show()

In [ ]:
import matplotlib.patches as patches
#xranges=((600,700),(575,675),(575,700))
#yranges=((900,1000),(775,875),(1375,1475))

alg=mgr.GetClusterAlg(3)

dm=mgr.DataManager()
ref_data = dm.Data(3)
tickscore0_y=[]
tickscore0_x=[]

score0_v = alg.TimeBinnedScore0Mean()
for idx in xrange(score0_v.size()):
    v = score0_v[idx]
    tickscore0_y.append(v)
    tickscore0_x.append(idx*1 + alg.TimeBinMin())
        
tickscore1_y=[]
tickscore1_x=[]
score1_v = alg.TimeBinnedScore1Mean()
for idx in xrange(score1_v.size()):
    v = score1_v[idx]
    tickscore1_y.append(v)
    tickscore1_x.append(idx*1 + alg.TimeBinMin())

tickscore0_x = np.array(tickscore0_x)
tickscore0_y = np.array(tickscore0_y)
tickscore1_x = np.array(tickscore1_x)
tickscore1_y = np.array(tickscore1_y)
ymin = tickscore0_y.min()
ymax = tickscore0_y.max()
if ymin > tickscore1_y.min(): ymin = tickscore1_y.min()
if ymax < tickscore1_y.max(): ymax = tickscore1_y.max()
    
fig,ax = plt.subplots(figsize=(24,8),facecolor='w')
plt.plot(tickscore0_x,tickscore0_y,marker='o',linestyle='-',color='red')
plt.plot(tickscore1_x,tickscore1_y,marker='o',linestyle='-',color='blue')

#minimum1_v = ref_data._time_binned_score1_minidx_v
#for idx in xrange(minimum1_v.size()):
#    xval = tickscore1_x[minimum1_v[idx]]
#    plt.plot([xval,xval],[ymin,ymax],marker='',linestyle='-',color='blue')

#minimum0_v = ref_data._time_binned_score0_minidx_v
#for idx in xrange(minimum0_v.size()):
#    xval = tickscore0_x[minimum0_v[idx]]
#    plt.plot([xval,xval],[ymin,ymax],marker='',linestyle='--',color='red')

minimum_v = alg.TimeBinnedScoreMinIndex()
for idx in xrange(minimum_v.size()):
    xval = tickscore0_x[minimum_v[idx]]
    plt.plot([xval,xval],[ymin,ymax],marker='',linestyle='--',color='black',linewidth=2)
    
ax.set_ylim(ymin,ymax)
ax.set_xlim(tickscore0_x.min(),tickscore0_x.max())
#ax.set_xlim(475,550)
plt.show()

In [ ]:
import matplotlib.patches as patches
#xranges=((600,700),(575,675),(575,700))
#yranges=((900,1000),(775,875),(1375,1475))
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(20,20),facecolor='w')
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    
    ymin,ymax = (np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    xmin,xmax = (np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)    
    
    #xmin,xmax=(400,700) # entry 10
    #xmin,xmax=(130,180) # entry 8
    
    #ymin,ymax = (740,760) #entry 8 plane 0
    #ymin,ymax = (480,550) #entry 8 plane 1
    #ymin,ymax = (880,930) #entry 8 plane 2
    
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(xmin,xmax)
    ix=-1

    #for cv_ctor_lines in cv_ctor_lines_v:
    #
    #    ix+=1
    #    color=colormap[ix]
    #    for line in cv_ctor_lines:
    #        pass#plt.plot([0,900],[line.y(0),line.y(900)],alpha=0.7,color=color)
    #    
    #for inter_pt in cv_inter_pts_v[plane]:
    #    pass#plt.plot(inter_pt.x,inter_pt.y,'o',markersize=10,color=color,alpha=0.9)   
        
    alg = mgr.GetClusterAlg(3)
    
    cv=larbysimg.Manager().GetClusterAlg(3)
    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    
    #Getting Defect Points
    defect_data=dm.Data(1)
    compound_v = defect_data._raw_cluster_vv[plane]
    for compound in compound_v.get_cluster():
        for defect in compound.get_defects():
            pt = defect._pt_defect
            plt.plot([pt.x],[pt.y],marker='o',color='blue',markersize=10)
    
    pca_data = dm.Data(2)
    for pt in pca_data.points(plane):
        plt.plot([pt.x],[pt.y],marker='o',color='red',markersize=10)
            
    ref_data   = dm.Data(3)
    print ref_data
    plane_scan = ref_data.get_plane_data(plane)

    for circle in plane_scan._circle_scan_v:
        c=patches.Circle((circle.center.x,circle.center.y),circle.radius,ec='cyan',alpha=0.05,fc='none',lw=10)
        ax.add_patch(c)
        
    minimum_v = alg.TimeBinnedScoreMinIndex()
    for idx in xrange(minimum_v.size()):
        xval = minimum_v[idx] + alg.TimeBinMin()
        plt.plot([xval,xval],[ymin,ymax],marker='',linestyle='--',color='magenta',linewidth=2)

    vtx_vv = ref_data.get_circle_vertex()
    
    vtx_idx=0
    for vtx_idx in xrange(len(vtx_vv)):
        vtx = vtx_vv[vtx_idx][plane]
    
    
    #for vtx_v in vtx_vv:
        #print vtx_v
        #vtx = vtx_v[plane]
        plt.plot([vtx.center.x],[vtx.center.y],marker='$\star$',color='yellow',markersize=24)
        
    ax=plt.gca()
    #ax.set_aspect(1.0)
    #plt.savefig()
    plt.show()
    



In [ ]:
#New VertexCluster

data=dm.Data(4)
#Simply a list of VertexCluster (i.e. vertex-wise list of clusters)
all_vtx_cluster=data._vtx_cluster_v
print "There are ",len(all_vtx_cluster),"vertexs found in this event. Note that this is vertex-wise"
#print all_vtx_cluster
################
for vtx_idx in xrange(len(all_vtx_cluster)):
        
    vtx_particles = all_vtx_cluster[vtx_idx]
    for plane in xrange(3):
        
        vtx2d = dm.Data(3).get_circle_vertex(vtx_particles.id())[plane]
        xmin,xmax = (vtx2d.center.x-200,vtx2d.center.x+200)
        ymin,ymax = (vtx2d.center.y-200,vtx2d.center.y+200)
        
        vtx_clusters =  all_vtx_cluster[vtx_idx]
        clusters     =  vtx_clusters.get_clusters(plane)
        vtx_3d       =  vtx_clusters.get_vertex()
        vtx_2d       =  vtx_3d.vtx2d_v
        vtx_2d_plane =  vtx_2d[plane]
        
        print 'Vertex ID',vtx_clusters.id(),'plane',plane

        #Get the image
        fig,ax = plt.subplots(figsize=(20,20),facecolor='w')
        shape_img = img_v[plane]
        shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
        #shape_img[:,::-1]=shape_img
        plt.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)
        ax.set_ylim(np.max(nz_pixels[0])-10,np.min(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
            
        circles      =  all_vtx_cluster[vtx_idx].get_circle_vertex(plane)
        center       =  circles.center
        for cluster in clusters:
            ctor=cluster._ctor
            pts=np.array([[pt.x,pt.y] for pt in ctor])
            plt.plot(pts[:,0],pts[:,1],'-o')
            plt.plot(center.x, center.y, marker='$\star$',color='pink',markersize=30)
        ax.set_title("Plane %i"%plane)
        plt.xlim(xmin,xmax)
        plt.ylim(ymin,ymax)
        plt.show()

In [ ]:
mgr=larbysimg.Manager()
dm=mgr.DataManager()

data=dm.Data(5)
for vtx_particles in data.get_vertex_clusters():
    for plane in xrange(3):
        print 'Vertex ID',vtx_particles.id(),'plane',plane

        vtx2d = dm.Data(3).get_circle_vertex(vtx_particles.id())[plane]
        xmin,xmax = (vtx2d.center.x-200,vtx2d.center.x+200)
        ymin,ymax = (vtx2d.center.y-200,vtx2d.center.y+200)
        
        fig,ax=plt.subplots(figsize=(12,12))
        #Get the image
        shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
        #shape_img[:,::-1]=shape_img
        plt.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        #ax.set_ylim(650,700)
        #ax.set_xlim(275,325)
        
        for compound in vtx_particles.get_cluster(plane):
            for atom in compound.get_atoms():
                pts = np.array([[pt.x,pt.y] for pt in atom._ctor])
                plt.plot(pts[:,0],pts[:,1],'-o')
            for defect in compound.get_defects():
                ax.plot([0,900],[defect._split_line.y(0),defect._split_line.y(900)])

        ax.set_aspect(1.0)
        ax.set_title("ALL Contours")
        plt.xlim(xmin,xmax)
        plt.ylim(ymin,ymax)
        plt.show()

In [ ]:
mgr=larbysimg.Manager()
dm=mgr.DataManager()
data=dm.Data(7)
print data.get_vertex_clusters().size()
for vtx_dqdx in data.get_vertex_clusters():
    for plane in xrange(3):
        print 'Vertex ID',vtx_dqdx.id(),'plane',plane
        
        vtx2d = dm.Data(3).get_circle_vertex(vtx_particles.id())[plane]
        xmin,xmax = (vtx2d.center.x-200,vtx2d.center.x+200)
        ymin,ymax = (vtx2d.center.y-200,vtx2d.center.y+200)
        
        fig,ax=plt.subplots(figsize=(12,12))
        #Get the image
        shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
        #shape_img[:,::-1]=shape_img
        plt.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        vtx_particles = dm.Data(5).get_vertex_cluster(vtx_dqdx.id())
        
        for compound in vtx_particles.get_cluster(plane):
            part_dqdx = vtx_dqdx.get_cluster(plane)[compound.id()]
            print 'Particle ID',part_dqdx.id(),compound.id()
            
            for atom_idx in xrange(part_dqdx.num_atoms()):
                atom_start = part_dqdx.atom_start_pt(atom_idx)
                atom_id    = part_dqdx.atom_id(atom_idx)
                atom_ctor  = compound.get_atom(atom_id)._ctor

                print 'start x:',atom_start.x,
                print 'start y:',atom_start.y,
                print 'atom id:',atom_id
                
                pts = np.array([[pt.x,pt.y] for pt in atom_ctor])
                plt.plot(pts[:,0],pts[:,1],'-o',color=colormap[atom_idx])
                
                plt.plot([atom_start.x],[atom_start.y],marker='$\star$',
                         color=colormap[atom_idx],markersize=32, alpha=0.25)
                
        ax.set_aspect(1.0)
        ax.set_title("ALL Contours")
        plt.xlim(xmin,xmax)
        plt.ylim(ymin,ymax)
        plt.show()
        
        for part_dqdx in vtx_dqdx.get_cluster(plane):            

            dqdx = part_dqdx.dqdx()
            start_index_v = part_dqdx.atom_start_index_array()
            range_v = []
            for i in xrange(start_index_v.size()):
                if not i: continue
                range_v.append((start_index_v[i-1],start_index_v[i]))
            range_v.append((start_index_v.back(),dqdx.size()-1))
            if not(range_v): continue
            
            fig,ax = plt.subplots(figsize=(12,6),facecolor='w')
            for i in xrange(len(range_v)):
                start,end = range_v[i]
                data_y = []
                data_x = []
                while 1:
                    data_y.append(dqdx[start])
                    data_x.append(start)
                    start +=1
                    if start == end: break
                print range_v[i],'/',dqdx.size()
                plt.plot(data_x,data_y,marker='o',color=colormap[i])
            plt.show()
            print 'particle',part_dqdx.id(),'done'